In [5]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

num_inputs = 2
num_examples  = 1000

true_w = [2,-3.4]
true_b = 4.2

x = nd.random_normal(shape = (num_examples, num_inputs))
y = true_w[0] * x[:, 0] + true_w[1] * x[:,1] + true_b
y+= .01 * nd.random_normal(shape = y.shape)

数据读取,这里使用data模块来读取

In [6]:
batch_size = 10
dataset = gluon.data.ArrayDataset(x, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle = True)

读取和autograd文件里的一致

In [7]:
for data, label in data_iter:
    print(data, label)
    break


[[ 0.4854497   0.07510404]
 [ 0.23451622 -1.2184286 ]
 [ 0.7313352   0.8966576 ]
 [ 0.2659449  -0.7830153 ]
 [ 0.9298198   0.5596285 ]
 [-0.77552176 -0.43291265]
 [-0.569918    0.8948051 ]
 [-0.16282491  0.6836102 ]
 [-0.63778454 -0.05813766]
 [-1.2732902   1.4936081 ]]
<NDArray 10x2 @cpu(0)> 
[ 4.9133105   8.818591    2.61524     7.389907    4.166425    4.107523
  0.02080577  1.5605415   3.1055021  -3.4298012 ]
<NDArray 10 @cpu(0)>


定义模型: gluon提供了大量定制好的层, 我们只需要调用层来构建模型.例如线性模型为Dense层,构造模型最简单的办法是利用Sequential把所有层串起来,首先定义一个新模型

In [8]:
net = gluon.nn.Sequential()

然后加入一个Dense层,它需要定义输出节点

In [9]:
net.add(gluon.nn.Dense(1))
net

Sequential(
  (0): Dense(None -> 1, linear)
)

In [10]:
net.initialize()

In [11]:
square_loss = gluon.loss.L2Loss()

In [12]:
trainer = gluon.Trainer(
net.collect_params(), 'sgd', {'learning_rate': 0.1})

训练: 不再调用SGD,用train.step来更新模型

In [13]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print('Epoch %d, average loss: %f'%(e, total_loss/num_examples))

Epoch 0, average loss: 0.884086
Epoch 1, average loss: 0.000050
Epoch 2, average loss: 0.000050
Epoch 3, average loss: 0.000050
Epoch 4, average loss: 0.000050


In [ ]:
比较学到的和真实的模型,先从net拿到需要的层,然后访问其权重和位移

In [14]:

dense = net[0]
true_w,dense.weight.data()


([2, -3.4], 
 [[ 1.9996793 -3.4001975]]
 <NDArray 1x2 @cpu(0)>)

In [15]:
true_b,dense.bias.data()

(4.2, 
 [4.199619]
 <NDArray 1 @cpu(0)>)

In [18]:
dense.weight??